In [10]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

import pandas as pd

In [2]:
X_train_gendered = pd.read_csv("X_train_gendered.csv")
X_dev_gendered = pd.read_csv("X_dev_gendered.csv")
X_test_gendered = pd.read_csv("X_test_gendered.csv")
y_train_gendered = pd.read_csv("y_train_gendered.csv")
y_dev_gendered = pd.read_csv("y_dev_gendered.csv")
y_test_gendered = pd.read_csv("y_test_gendered.csv")

X_train_not_gendered = pd.read_csv("X_train_not_gendered.csv")
X_dev_not_gendered = pd.read_csv("X_dev_not_gendered.csv")
X_test_not_gendered = pd.read_csv("X_test_not_gendered.csv")
y_train_not_gendered = pd.read_csv("y_train_not_gendered.csv")
y_dev_not_gendered = pd.read_csv("y_dev_not_gendered.csv")
y_test_not_gendered = pd.read_csv("y_test_not_gendered.csv")

In [29]:
def complete_matrix(df, gendered, include_gender = False):
    vectorizer = TfidfVectorizer(stop_words='english')
    
    tvec = vectorizer.fit_transform(df['body'].values.astype('U'))

    
    tf_idf_df = pd.DataFrame(tvec.toarray(), columns=vectorizer.get_feature_names())
    
    if gendered:
        original = df[["score", "num_comments"]]
    else:
        if not include_gender:
            original = df[["score", "num_comments", "age"]]
        else:
            original["gender"] = original["gender"].str.upper()
            original.replace({'gender': {"M": 1, "F": 0}})
            original = df[["score", "num_comments", "age", "gender"]]
        
    res = pd.concat([tf_idf_df, original], axis=1)
    
    return res


In [17]:
y_train_gendered.head(2)

,Unnamed: 0,verdict
0,5661,not the asshole
1,12868,asshole


In [18]:
def model_and_scores(train_x, train_y, dev_x, dev_y, test_x, test_y, include_gender = False):
    model = LogisticRegression(multi_class = "multinomial", max_iter = 500)

    complete_train_x = complete_matrix(train_x, include_gender)
    model.fit(complete_train_x, train_y.verdict)
    
    complete_dev_x = complete_matrix(dev_x, include_gender)
    dev_scores = accuracy_score(model.predict(complete_dev_x), dev_y.verdict)
    print(f"Dev Scores: {dev_scores}")
    
    complete_test_x = complete_matrix(test_x, include_gender)
    test_scores = accuracy_score(model.predict(complete_test_x), test_y.verdict)
    print(f"Test Scores: {test_scores}")


In [33]:
model_and_scores(X_train_not_gendered, y_train_not_gendered, X_dev_not_gendered, y_dev_not_gendered, X_test_not_gendered, y_test_not_gendered, False)

KeyError: "['age'] not in index"

In [ ]:
model_and_scores(X_train_gendered, y_train_gendered, X_dev_gendered, y_dev_gendered, X_test_gendered, y_test_gendered, True)
model_and_scores(X_train_gendered, y_train_gendered, X_dev_gendered, y_dev_gendered, X_test_gendered, y_test_gendered, True, True)

In [32]:
X_train_not_gendered

,Unnamed: 0,Unnamed: 0.1,id,timestamp,title,body,edited,score,num_comments,is_asshole,text
0,77658,85277,ecmy36,1.576724e+09,AITA for ruining a family tradition?,"So, the title may be a bit of an overstatement...",False,4,10.0,0,"AITA for ruining a family tradition? So, the t..."
1,97785,731,4c17ki,1.459000e+09,"AITA My friend blocked me for being well, an ass.",**Preface (feel free to skip):** So I came acr...,False,11,22.0,1,"AITA My friend blocked me for being well, an a..."
2,76961,84483,ebbx1u,1.576481e+09,WIBTA if I told my friend that I wanted to han...,Met a cool dude when I moved to a new area on ...,False,10,19.0,0,WIBTA if I told my friend that I wanted to han...
3,40256,43222,c9dj6h,1.562315e+09,AITA for wanting a mother and her kid to leave...,I have been seeing the woman for a few months....,False,7,11.0,0,AITA for wanting a mother and her kid to leave...
4,64659,70567,djvuvj,1.571442e+09,WIBTA if I report my friend to her job that sh...,I have a friend who is screwing up her life. S...,1571442679.0,4,53.0,0,WIBTA if I report my friend to her job that sh...
...,...,...,...,...,...,...,...,...,...,...,...
102664,21656,22933,b8msd0,1.554229e+09,AITA for turning in my ex girlfriend for plagi...,Before Reading this post I know that academic ...,False,4,21.0,1,AITA for turning in my ex girlfriend for plagi...
102665,118528,2963,8ks6g0,1.526818e+09,AITA For stealing a friend's meat at a wedding?,I am 38F. I have been married for 2 years to m...,False,33,51.0,1,AITA For stealing a friend's meat at a wedding...
102666,10486,10894,ahwrol,1.547985e+09,AITA for being hurt that my parents gave my si...,I’ll try to keep this as brief as I can while ...,False,14,14.0,0,AITA for being hurt that my parents gave my si...
102667,33679,36003,bw9dpa,1.559560e+09,AITA for hiring a girl to be in a relationship...,Recently a close friend of mine received news ...,False,33,74.0,1,AITA for hiring a girl to be in a relationship...
